# Clean Excel data from Dalinde Hospital

In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np
import sklearn
import os
from tqdm.auto import tqdm, trange

_________________

## Informes de consumo 2018-2019

In [2]:
# Retrieve names of files
data_files = os.listdir('data/')
data_files = [file for file in data_files if 'Informe de consumo por areas Tuxpan' in file]
data_files

['Informe de consumo por areas Tuxpan enero 2018.xlsx',
 'Informe de consumo por areas Tuxpan Febrero 2018.xlsx',
 'Informe de consumo por areas Tuxpan Diciembre 2018.xlsx',
 'Informe de consumo por areas Tuxpan Agosto 2019.xlsx',
 'Informe de consumo por areas Tuxpan Septiembre 2019.xlsx',
 'Informe de consumo por areas Tuxpan Noviembre 2018.xlsx',
 'Informe de consumo por areas Tuxpan Junio 2019.xlsx',
 'Informe de consumo por areas Tuxpan mayo 2018.xlsx',
 'Informe de consumo por areas Tuxpan Mayo 2019.xlsx',
 'Informe de consumo por areas Tuxpan Marzo 2019.xlsx',
 'Informe de consumo por areas Tuxpan Noviembre 2019.xlsx',
 'Informe de consumo por areas Tuxpan Julio 2019.xlsx',
 'Informe de consumo por areas Tuxpan Septiembre 2018.xlsx',
 'Informe de consumo por areas Tuxpan marzo 2018.xlsx',
 'Informe de consumo por areas Tuxpan junio 2018.xlsx',
 'Informe de consumo por areas Tuxpan Julio 2018.xlsx',
 'Informe de consumo por areas Tuxpan Octubre 2019.xlsx',
 'Informe de consumo po

In [3]:
# Months dictionary
months = {'ENERO':'01', 'FEBRERO':'02', 'MARZO':'03', 'ABRIL':'04', 'MAYO':'05', 'JUNIO':'06', 'JULIO':'07', 'AGOSTO':'08', 'SEPTIEMBRE':'09', 'OCTUBRE':'10', 'NOVIEMBRE':'11', 'DICIEMBRE':'12'}
# Empty DataFrame
data = pd.DataFrame()

for i in trange(len(data_files), desc = 'Processing files'):
      
    filename = data_files[i] # Name of file
    
    # Retreive MONTH and YEAR from filename
    month = months[filename.split(' ')[6].upper()]
    year = filename.split(' ')[7].split('.')[0]
    # Set date with first day of the month
    date = pd.to_datetime(year+month, format='%Y%m')
    # Assign last day of the month
    date = pd.to_datetime(year+month+str(date.days_in_month), format='%Y%m%d')
    
    print('Processing data from '+str(date)) # For tracking progress
    
    # Load file
    df = pd.read_excel('data/'+filename, header=5)
    
    # Fill ÁREA and DEPARTAMENTO columns since the only appear once
    df[["ÁREA","DEPARTAMENTO"]] = df[["ÁREA","DEPARTAMENTO"]].fillna(method='ffill')
    # Drop all rows without a CAPTURA id, which are thrash 
    df = df.loc[df['CAPTURA'].dropna().index, :]
    # Retrieve items which DEPARTAMENTO =='PATENTE'
    df = df.loc[df['DEPARTAMENTO'] == 'PATENTE', :]
    # Add FECHA column
    df["FECHA"] = date
    df = df.loc[:, ["FECHA", "DEPARTAMENTO", "CAPTURA", "CÓDIGO", "ARTÍCULO", "CANTIDAD", "COSTO UNITARIO","TOTAL NETO"]]
    
    # Concatenate df
    data = pd.concat([data, df], ignore_index=True)

# Sort values by date
data = data.sort_values(['FECHA','ARTÍCULO'], ignore_index=True)
# Write data to CSV file
data.to_csv('Informe de consumo 2018-2019.csv', index=False)
data

Processing data from 2018-01-31 00:00:00
Processing data from 2018-02-28 00:00:00
Processing data from 2018-12-31 00:00:00
Processing data from 2019-08-31 00:00:00
Processing data from 2019-09-30 00:00:00
Processing data from 2018-11-30 00:00:00
Processing data from 2019-06-30 00:00:00
Processing data from 2018-05-31 00:00:00
Processing data from 2019-05-31 00:00:00
Processing data from 2019-03-31 00:00:00
Processing data from 2019-11-30 00:00:00
Processing data from 2019-07-31 00:00:00
Processing data from 2018-09-30 00:00:00
Processing data from 2018-03-31 00:00:00
Processing data from 2018-06-30 00:00:00
Processing data from 2018-07-31 00:00:00
Processing data from 2019-10-31 00:00:00
Processing data from 2018-08-31 00:00:00
Processing data from 2019-12-31 00:00:00
Processing data from 2019-04-30 00:00:00
Processing data from 2019-01-31 00:00:00
Processing data from 2018-10-31 00:00:00
Processing data from 2018-04-30 00:00:00
Processing data from 2019-02-28 00:00:00
Processing data 

,FECHA,DEPARTAMENTO,CAPTURA,CÓDIGO,ARTÍCULO,CANTIDAD,COSTO UNITARIO,TOTAL NETO
0,2018-01-31,PATENTE,5235410.0,3300629.0,ACC 200MG C/20 TAB EFERV (ACETILCISTEINA),1.0,127.59,127.59
1,2018-01-31,PATENTE,5233949.0,3304626.0,ACC 600MG TAB EFERV (ACETILCISTEINA),1.0,252.48,252.48
2,2018-01-31,PATENTE,5235566.0,3304626.0,ACC 600MG TAB EFERV (ACETILCISTEINA),1.0,252.48,252.48
3,2018-01-31,PATENTE,5245250.0,3302703.0,ACELER CO 2% CREMA 15G(ACIDO FUSIDICO/BETAMETA...,1.0,206.73,206.73
4,2018-01-31,PATENTE,5265293.0,3302703.0,ACELER CO 2% CREMA 15G(ACIDO FUSIDICO/BETAMETA...,1.0,206.73,206.73
...,...,...,...,...,...,...,...,...
104890,2019-12-31,PATENTE,6183131.0,3300140.0,ZYPREXA IM 10MG AMP 2ML (OLANZAPINA),1.0,292.56,292.56
104891,2019-12-31,PATENTE,6173743.0,3300140.0,ZYPREXA IM 10MG AMP 2ML (OLANZAPINA),1.0,292.56,292.56
104892,2019-12-31,PATENTE,6165542.0,3300182.0,ZYPREXA ZYDIS 10MG C/14 TABS (OLANZAPINA),1.0,1710.24,1710.24
104893,2019-12-31,PATENTE,6201334.0,3600131.0,ZYVOXAM TABLETAS 600MG C/10 (LINEZOLID),1.0,7607.83,7607.83


_________________

## Requisiciones vs órdenes de compra

In [4]:
# Retrieve names of files
data_files = os.listdir('data/')
data_files = [file for file in data_files if 'requisiciones vs ordenes de compra' in file]
data_files

['requisiciones vs ordenes de compra 2019.xlsx',
 'requisiciones vs ordenes de compra 2018 (1).xlsx']

In [5]:
# Empty DataFrame
data = pd.DataFrame()

for i in trange(len(data_files), desc = 'Processing files'):

    filename = data_files[i] # Name of file
    print('Processing data from '+filename) # For tracking progress
    
    # Load file
    df = pd.read_excel('data/'+filename, header=[4,5])
    # Restore columns names
    df.columns = [(col[0] if 'Unnamed' in col[1] else col[0]+' '+col[1]) for col in df.columns.to_flat_index()]
    df = df.rename(columns={'DESCRIPCIÓN SALIDAS':'SALIDAS'})
    
    # Concatenate df
    data = pd.concat([data, df], ignore_index=True)

# Drop unused columns
data = data.loc[:, data.columns[:-2]]
# Standirize date format
data[['FECHA','FECHA PEDIDO','FECHA ENTREGA']] = data[['FECHA','FECHA PEDIDO','FECHA ENTREGA']].apply(pd.to_datetime, format='%d-%m-%Y')

# Sort values by REQUISICIÓN id
data = data.sort_values(['REQUISICIÓN','FECHA'], ignore_index=True)
# Write data to CSV file
data.to_csv('Requisiciones vs ordenes de compra 2018-2019.csv', index=False)
data

Processing data from requisiciones vs ordenes de compra 2019.xlsx
Processing data from requisiciones vs ordenes de compra 2018 (1).xlsx



,REQUISICIÓN,FECHA,ORDEN COMPRA,FECHA PEDIDO,FECHA ENTREGA,PROVEEDOR,ARTÍCULO,DESCRIPCIÓN,SALIDAS,CANTIDAD REQUERIDA,CANTIDAD PEDIDA
0,52666,2018-01-02,257941.0,2018-01-03,2018-01-10,"NADRO, S.A.P.I. DE C.V.",2900110,#DEXERYL CREMA EMOLIENTE 250G,199.0,1.0,1
1,52666,2018-01-02,257941.0,2018-01-03,2018-01-10,"NADRO, S.A.P.I. DE C.V.",2900233,#ISODINE 8G BUCOFARINGEO SOL 120ML,198.0,2.0,2
2,52666,2018-01-02,257941.0,2018-01-03,2018-01-10,"NADRO, S.A.P.I. DE C.V.",2900274,#VASELINE PURO 100G,196.0,2.0,2
3,52666,2018-01-02,257941.0,2018-01-03,2018-01-10,"NADRO, S.A.P.I. DE C.V.",2900493,VIVERA 2G 8 SOBRES LACTOBACILLUS RHAMNOSUS GG,197.0,2.0,2
4,52666,2018-01-02,257942.0,2018-01-03,2018-01-10,FARMACOS NACIONALES S.A. DE C.V. (FANASA),2900150,#BICARBONATO DE SODIO 100G,198.0,2.0,2
...,...,...,...,...,...,...,...,...,...,...,...
64167,66178,2019-12-30,315609.0,2019-12-30,2020-01-03,DISTRIBUIDORA MEDICA CODIMED S.A. DE C.V.,3800068,FISIOLOGICA 50 ML FCO. PISA,1282.0,1200.0,1200
64168,66178,2019-12-30,315609.0,2019-12-30,2020-01-03,DISTRIBUIDORA MEDICA CODIMED S.A. DE C.V.,3800066,FISIOLOGICA 500ML FCO. PISA,166.0,168.0,168
64169,66178,2019-12-30,315609.0,2019-12-30,2020-01-03,DISTRIBUIDORA MEDICA CODIMED S.A. DE C.V.,3800064,HARTMAN 1000ML FRASCO PISA,143.0,120.0,120
64170,66178,2019-12-30,315609.0,2019-12-30,2020-01-03,DISTRIBUIDORA MEDICA CODIMED S.A. DE C.V.,3800077,HARTMAN 500 ML FLEXOVAL SOLUCION PISA 4000088,78.0,72.0,72


_________________

## Informe órdenes incompletas

In [6]:
# Retrieve names of files
data_files = os.listdir('data/')
data_files = [file for file in data_files if 'Requisiciones ordenes incompletas' in file]
data_files

['Requisiciones ordenes incompletas 2018.xlsx',
 'Requisiciones ordenes incompletas 2019.xlsx']

In [7]:
# Empty DataFrame
data = pd.DataFrame()

for i in trange(len(data_files), desc = 'Processing files'):

    filename = data_files[i] # Name of file
    print('Processing data from '+filename) # For tracking progress
    
    # Load file
    df = pd.read_excel('data/'+filename, header=[4,5])
    # Restore columns names
    df.columns = [(col[0] if 'Unnamed' in col[1] else col[0]+' '+col[1]) for col in df.columns.to_flat_index()]
    
    # Concatenate df
    data = pd.concat([data, df], ignore_index=True)

# Drop unused columns
data = data.loc[:, np.append(data.columns[:-3], data.columns[-2])]
# Standirize date format
data[['FECHA PEDIDO','FECHA ENTREGA','FECHA SOLICITUD','FECHA RECEPCION']] = data[['FECHA PEDIDO','FECHA ENTREGA','FECHA SOLICITUD','FECHA RECEPCION']].apply(pd.to_datetime, format='%d-%m-%Y', errors='coerce')

# Write data to CSV file
data.to_csv('Requisiciones ordenes incompletas 2018-2019.csv', index=False)
data

Processing data from Requisiciones ordenes incompletas 2018.xlsx
Processing data from Requisiciones ordenes incompletas 2019.xlsx



,PROVEEDOR,PEDIDO,FECHA PEDIDO,FECHA ENTREGA,REQUISICION,FECHA SOLICITUD,C. COSTOS,ARTÍCULO,DESCRIPCIÓN,FACTURA,FECHA RECEPCION,CANTIDAD REQUISICION,CANTIDAD SOLICITADA,CANTIDAD RECIBIDA,%
0,"ABASTECEDOR TERAPEUTICO, S.A. DE C.V. (ATSA)",258679.0,2018-01-15,2018-01-22,52889.0,2018-01-15,MATERIAL DE CURACION,1900240,#EXTENSION K/750 50CM MEDEX MX450FL,NaN,NaT,250.0,250.0,0,0.0
1,"ABASTECEDOR TERAPEUTICO, S.A. DE C.V. (ATSA)",262080.0,2018-02-26,2018-03-05,53711.0,2018-02-26,MATERIAL DE CURACION,1900947,#EXTENSION MINIVOLUMEN 536035 1.0ML VOL RESIDU...,NaN,NaT,50.0,50.0,0,0.0
2,NaN,NaN,NaT,NaT,NaN,NaT,NaN,1900979,#KIT P/MONITOREO MX9505T TRANSDUCTOR MEDEX,NaN,NaT,10.0,10.0,0,0.0
3,NaN,NaN,NaT,NaT,NaN,NaT,NaN,1901413,#KIT P/MONITOREO MX9604A TRANSDUCTOR MEDEX,NaN,NaT,20.0,20.0,0,0.0
4,"ABASTECEDOR TERAPEUTICO, S.A. DE C.V. (ATSA)",268297.0,2018-05-21,2018-05-28,55285.0,2018-05-21,MATERIAL DE CURACION,1900313,#LLAVE 3 VIAS S/EXTENSION MX5311L MEDEX,NaN,NaT,250.0,250.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7105,"VITASANITAS, S.A. DE C.V.",303167.0,2019-08-06,2019-08-07,63296.0,2019-08-06,MEDICAMENTOS,4200011,SMOF KABIVEN 1100KCAL CENTRAL (TRICAMARA NUTRC...,NaN,NaT,2.0,2.0,0,0.0
7106,"VITASANITAS, S.A. DE C.V.",309257.0,2019-10-18,2019-10-21,0.0,NaT,MEDICAMENTOS,4200011,SMOF KABIVEN 1100KCAL CENTRAL (TRICAMARA NUTRC...,NaN,NaT,0.0,10.0,0,0.0
7107,"VITASANITAS, S.A. DE C.V.",309763.0,2019-10-23,2019-10-24,64862.0,2019-10-23,MEDICAMENTOS,4200011,SMOF KABIVEN 1100KCAL CENTRAL (TRICAMARA NUTRC...,NaN,NaT,2.0,2.0,0,0.0
7108,"VITASANITAS, S.A. DE C.V.",310438.0,2019-11-01,2019-11-04,65015.0,2019-11-01,MEDICAMENTOS,3800003,ALBUMINA HUMANA AL 20% FCO 50ML,NaN,NaT,6.0,6.0,0,0.0


There seems to be many Na values. Let's count them.

In [8]:
data.isna().sum()

PROVEEDOR               4761
PEDIDO                  4761
FECHA PEDIDO            4761
FECHA ENTREGA           4761
REQUISICION             4761
FECHA SOLICITUD         5207
C. COSTOS               4761
ARTÍCULO                   0
DESCRIPCIÓN                0
FACTURA                 5640
FECHA RECEPCION         5640
CANTIDAD REQUISICION     637
CANTIDAD SOLICITADA      637
CANTIDAD RECIBIDA          0
%                          0
dtype: int64

Only ARTÍCULO, DESCRIPCIÓN, CANTIDAD RECIBIDA and % are missing no values. This is a good sign that the data is not corrupted.<br>
After exploring the data we concluded that we should not fill any Na values in the FACTURA and FECHA RECEPCION columns, as they serve as an indicator for orders that were not received.<br>
We also spotted the pattern that all rows with 'REQUISICION'==0 are missing 'FECHA SOLICITUD'. We most explore this further in the data analysis.

In [9]:
data[data['REQUISICION']==0]['FECHA SOLICITUD'].value_counts()

Series([], Name: FECHA SOLICITUD, dtype: int64)

As mentioned before, FACTURA and FECHA RECEPCION columns serve as an indicator for orders that were not received. We confirm it by noticing that fulfillment ('%') of every order missing these columns is always 0%.

In [10]:
data[data['FECHA RECEPCION'].isna()]['%'].value_counts()

0.0    5640
Name: %, dtype: int64

All rows that are missing 'CANTIDAD REQUISICION' and 'CANTIDAD SOLICITADA' are missing the first 7 columns too.

In [11]:
data[data['CANTIDAD REQUISICION'].isna()].isna().sum()

PROVEEDOR               637
PEDIDO                  637
FECHA PEDIDO            637
FECHA ENTREGA           637
REQUISICION             637
FECHA SOLICITUD         637
C. COSTOS               637
ARTÍCULO                  0
DESCRIPCIÓN               0
FACTURA                   0
FECHA RECEPCION           0
CANTIDAD REQUISICION    637
CANTIDAD SOLICITADA     637
CANTIDAD RECIBIDA         0
%                         0
dtype: int64

After this analysis we conclude that:<br>
1) There data for the columns ['PROVEEDOR','PEDIDO','FECHA PEDIDO','FECHA ENTREGA','REQUISICION','C. COSTOS'] only appear once per order. The consecutive rows belonging to the same order are missing these values.<br>
2) The data for the columns ['CANTIDAD REQUISICION','CANTIDAD SOLICITADA'] only appear once per article in all orders, meaning that the consecutive rows belonging to the same order and article are missing these values.<br>
3) 'FECHA SOLICITUD' also appears once per order. However, all rows with 'REQUISICION'==0 are intentionally missing 'FECHA SOLICITUD', so we should keep it like that to not corrupt the data.<br>
We will fill missing values according to the described behavours.

In [12]:
# Create a copy of the data
temp = data.copy()
# Fill in Na values from point 1)
temp[['PROVEEDOR','PEDIDO','FECHA PEDIDO','FECHA ENTREGA','REQUISICION','C. COSTOS']] = temp[['PROVEEDOR','PEDIDO','FECHA PEDIDO','FECHA ENTREGA','REQUISICION','C. COSTOS']].fillna(method='ffill')
# Fill in Na values from point 2)
temp[['CANTIDAD REQUISICION','CANTIDAD SOLICITADA']] = temp[['CANTIDAD REQUISICION','CANTIDAD SOLICITADA']].fillna(method='ffill')
# Sanity check
temp.isna().sum()

PROVEEDOR                  0
PEDIDO                     0
FECHA PEDIDO               0
FECHA ENTREGA              0
REQUISICION                0
FECHA SOLICITUD         5207
C. COSTOS                  0
ARTÍCULO                   0
DESCRIPCIÓN                0
FACTURA                 5640
FECHA RECEPCION         5640
CANTIDAD REQUISICION       0
CANTIDAD SOLICITADA        0
CANTIDAD RECIBIDA          0
%                          0
dtype: int64

In [13]:
# Fill in Na values from point 3)
temp['FECHA SOLICITUD'] = pd.concat([temp[temp['REQUISICION']!=0]['FECHA SOLICITUD'].fillna(method='ffill'), temp[temp['REQUISICION']==0]['FECHA SOLICITUD']]).sort_index()
# Sanity check
temp[temp['REQUISICION']==0].equals(temp[temp['FECHA SOLICITUD'].isna()])

True

We see that every row with 'REQUISICION'==0 is still missing 'FECHA SOLICITUD', which is what we wanted.

In [14]:
temp.isna().sum()

PROVEEDOR                  0
PEDIDO                     0
FECHA PEDIDO               0
FECHA ENTREGA              0
REQUISICION                0
FECHA SOLICITUD         3396
C. COSTOS                  0
ARTÍCULO                   0
DESCRIPCIÓN                0
FACTURA                 5640
FECHA RECEPCION         5640
CANTIDAD REQUISICION       0
CANTIDAD SOLICITADA        0
CANTIDAD RECIBIDA          0
%                          0
dtype: int64

Now that our dataset is in better shape we save it in an csv file.

In [16]:
# Write data to CSV file
temp.to_csv('Requisiciones ordenes incompletas 2018-2019.csv', index=False)
temp

,PROVEEDOR,PEDIDO,FECHA PEDIDO,FECHA ENTREGA,REQUISICION,FECHA SOLICITUD,C. COSTOS,ARTÍCULO,DESCRIPCIÓN,FACTURA,FECHA RECEPCION,CANTIDAD REQUISICION,CANTIDAD SOLICITADA,CANTIDAD RECIBIDA,%
0,"ABASTECEDOR TERAPEUTICO, S.A. DE C.V. (ATSA)",258679.0,2018-01-15,2018-01-22,52889.0,2018-01-15,MATERIAL DE CURACION,1900240,#EXTENSION K/750 50CM MEDEX MX450FL,NaN,NaT,250.0,250.0,0,0.0
1,"ABASTECEDOR TERAPEUTICO, S.A. DE C.V. (ATSA)",262080.0,2018-02-26,2018-03-05,53711.0,2018-02-26,MATERIAL DE CURACION,1900947,#EXTENSION MINIVOLUMEN 536035 1.0ML VOL RESIDU...,NaN,NaT,50.0,50.0,0,0.0
2,"ABASTECEDOR TERAPEUTICO, S.A. DE C.V. (ATSA)",262080.0,2018-02-26,2018-03-05,53711.0,2018-02-26,MATERIAL DE CURACION,1900979,#KIT P/MONITOREO MX9505T TRANSDUCTOR MEDEX,NaN,NaT,10.0,10.0,0,0.0
3,"ABASTECEDOR TERAPEUTICO, S.A. DE C.V. (ATSA)",262080.0,2018-02-26,2018-03-05,53711.0,2018-02-26,MATERIAL DE CURACION,1901413,#KIT P/MONITOREO MX9604A TRANSDUCTOR MEDEX,NaN,NaT,20.0,20.0,0,0.0
4,"ABASTECEDOR TERAPEUTICO, S.A. DE C.V. (ATSA)",268297.0,2018-05-21,2018-05-28,55285.0,2018-05-21,MATERIAL DE CURACION,1900313,#LLAVE 3 VIAS S/EXTENSION MX5311L MEDEX,NaN,NaT,250.0,250.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7105,"VITASANITAS, S.A. DE C.V.",303167.0,2019-08-06,2019-08-07,63296.0,2019-08-06,MEDICAMENTOS,4200011,SMOF KABIVEN 1100KCAL CENTRAL (TRICAMARA NUTRC...,NaN,NaT,2.0,2.0,0,0.0
7106,"VITASANITAS, S.A. DE C.V.",309257.0,2019-10-18,2019-10-21,0.0,NaT,MEDICAMENTOS,4200011,SMOF KABIVEN 1100KCAL CENTRAL (TRICAMARA NUTRC...,NaN,NaT,0.0,10.0,0,0.0
7107,"VITASANITAS, S.A. DE C.V.",309763.0,2019-10-23,2019-10-24,64862.0,2019-10-23,MEDICAMENTOS,4200011,SMOF KABIVEN 1100KCAL CENTRAL (TRICAMARA NUTRC...,NaN,NaT,2.0,2.0,0,0.0
7108,"VITASANITAS, S.A. DE C.V.",310438.0,2019-11-01,2019-11-04,65015.0,2019-11-01,MEDICAMENTOS,3800003,ALBUMINA HUMANA AL 20% FCO 50ML,NaN,NaT,6.0,6.0,0,0.0
